# Teknik Binning Menggunakan Data Iris

Install Library Pendukung

In [ ]:
!pip install matplotlib seaborn
!pip install pymysql
!pip insinstalltall pymysql pandas
!psycopgpsycopg2all2all psypsycopg2copg2
!pip install mysql-connector-python
!pip install mysql-connector-python psycopg2-binary

ERROR: unknown command "insinstalltall" - maybe you meant "uninstall"
/bin/bash: line 1: psycopgpsycopg2all2all: command not found


Import Library Pendukung

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tabulate import tabulate

Data Iris Bersih

In [ ]:
import pandas as pd
from sklearn.datasets import load_iris

# Load dataset Iris
iris = load_iris()
df = pd.DataFrame(iris.data, columns=['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width'])
df['Species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
print(tabulate(df, headers='keys', tablefmt='github', showindex=False))

|   Sepal Length |   Sepal Width |   Petal Length |   Petal Width | Species    |
|----------------|---------------|----------------|---------------|------------|
|            5.1 |           3.5 |            1.4 |           0.2 | setosa     |
|            4.9 |           3   |            1.4 |           0.2 | setosa     |
|            4.7 |           3.2 |            1.3 |           0.2 | setosa     |
|            4.6 |           3.1 |            1.5 |           0.2 | setosa     |
|            5   |           3.6 |            1.4 |           0.2 | setosa     |
|            5.4 |           3.9 |            1.7 |           0.4 | setosa     |
|            4.6 |           3.4 |            1.4 |           0.3 | setosa     |
|            5   |           3.4 |            1.5 |           0.2 | setosa     |
|            4.4 |           2.9 |            1.4 |           0.2 | setosa     |
|            4.9 |           3.1 |            1.5 |           0.1 | setosa     |
|            5.4 |          

## Clustering Data Iris

In [ ]:
kolom_fitur = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']

# Loop untuk melakukan clustering per kolom
for fitur in kolom_fitur:
    # Ambil kolom dan ubah ke 2D array
    X = df[[fitur]].values
    # KMeans clustering
    kmeans = KMeans(n_clusters=4, random_state=42)
    clusters = kmeans.fit_predict(X)
    # Tambahkan hasil klaster numerik ke DataFrame
    cluster_col = fitur + 'Cluster'
    df[cluster_col] = clusters

# Tampilkan hanya kolom hasil cluster
print(tabulate(df[[
    'Sepal LengthCluster',
    'Sepal WidthCluster',
    'Petal LengthCluster',
    'Petal WidthCluster'
]], headers='keys', tablefmt='github', showindex=False))

|   Sepal LengthCluster |   Sepal WidthCluster |   Petal LengthCluster |   Petal WidthCluster |
|-----------------------|----------------------|-----------------------|----------------------|
|                     2 |                    3 |                     1 |                    1 |
|                     2 |                    2 |                     1 |                    1 |
|                     2 |                    0 |                     1 |                    1 |
|                     2 |                    0 |                     1 |                    1 |
|                     2 |                    3 |                     1 |                    1 |
|                     1 |                    3 |                     1 |                    1 |
|                     2 |                    0 |                     1 |                    1 |
|                     2 |                    0 |                     1 |                    1 |
|                     2 |               

## Diskritisasi Menggunakan K-Means

In [ ]:
def cluster_and_map(column_name, n_clusters=4):
    # Ambil kolom dan ubah ke bentuk 2D
    data = df[[column_name]].values
    # Jalankan KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(data)
    # Tambahkan kolom hasil klaster dan kategori huruf
    cluster_col = f"{column_name} Cluster"
    kategori_col = column_name  # Use the original column name as the base for the new categorical column
    df[cluster_col] = clusters
    kategori_map = {i: chr(97+i) for i in range(n_clusters)}  # {0: 'a', 1: 'b', ...}
    df[kategori_col + ' Category'] = df[cluster_col].map(kategori_map) # Create a new column for the categorical data

# Terapkan untuk keempat kolom
cluster_and_map('Sepal Length')
cluster_and_map('Sepal Width')
cluster_and_map('Petal Length')
cluster_and_map('Petal Width')

# Tampilkan hasil kategori huruf saja
print(tabulate(df[[
    'Sepal Length Category',
    'Sepal Width Category',
    'Petal Length Category',
    'Petal Width Category'
]], headers='keys', tablefmt='github', showindex=False))

| Sepal Length Category   | Sepal Width Category   | Petal Length Category   | Petal Width Category   |
|-------------------------|------------------------|-------------------------|------------------------|
| c                       | d                      | b                       | b                      |
| c                       | c                      | b                       | b                      |
| c                       | a                      | b                       | b                      |
| c                       | a                      | b                       | b                      |
| c                       | d                      | b                       | b                      |
| b                       | d                      | b                       | b                      |
| c                       | a                      | b                       | b                      |
| c                       | a                      | b          

### Menggunakan Naive Bayes

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report

# Select the categorical features generated from K-Means clustering and the target variable
# Use the existing DataFrame 'df' which contains these columns
categorical_features = [
    'Sepal Length Category',
    'Sepal Width Category',
    'Petal Length Category',
    'Petal Width Category'
]

# Create a new DataFrame with only the selected categorical features and the target variable 'Species'
df_categorical = df[categorical_features + ['Species']].copy()

# Encode the categorical features and the target variable
le = LabelEncoder()
X = df_categorical[categorical_features].apply(le.fit_transform)  # encode all feature columns
y = le.fit_transform(df_categorical['Species'])  # encode the target label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = CategoricalNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Akurasi:", accuracy_score(y_test, y_pred))
print("\nLaporan Klasifikasi:\n", classification_report(y_test, y_pred))

Akurasi: 0.9

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      0.67      0.80         9
           2       0.79      1.00      0.88        11

    accuracy                           0.90        30
   macro avg       0.93      0.89      0.89        30
weighted avg       0.92      0.90      0.90        30



### Menggunakan Decision Tree

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tabulate import tabulate

# Misalnya df_ef berisi hasil diskretisasi Equal Frequency atau hasil kategorisasi lainnya
# Contoh:
# df_ef = df[['Sepal Length EF Category', 'Sepal Width EF Category', 'Petal Length EF Category', 'Petal Width EF Category', 'Species']].copy()

# Rename agar seragam
df_dt = df.copy()

# Select the categorical features generated from K-Means clustering or other categorization
# Using the K-Means ' Category' columns
kategori_cols = [
    'Sepal Length Category',
    'Sepal Width Category',
    'Petal Length Category',
    'Petal Width Category'
]

# Encode fitur kategori huruf (a, b, c, d) menjadi angka
le = LabelEncoder()
# Apply fit_transform to each column separately to handle potential NaN values gracefully,
# or ensure no NaNs if using .apply() on the entire DataFrame.
# A safer way is to iterate or ensure the data is clean before encoding.
# Assuming no NaNs in these generated category columns based on the previous steps:
X = df_dt[kategori_cols].apply(le.fit_transform)


# Encode target label
y = LabelEncoder().fit_transform(df_dt['Species'])

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Buat dan latih model Decision Tree
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred = model.predict(X_test)
print("Akurasi Decision Tree:", accuracy_score(y_test, y_pred))
print("\nLaporan Klasifikasi:\n", classification_report(y_test, y_pred))

# Cetak struktur pohon keputusan
feature_names = X.columns.tolist()
tree_rules = export_text(model, feature_names=feature_names)
print("\nStruktur Pohon Keputusan:\n")
print(tree_rules)

Akurasi Decision Tree: 0.9

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.88      0.78      0.82         9
           2       0.83      0.91      0.87        11

    accuracy                           0.90        30
   macro avg       0.90      0.90      0.90        30
weighted avg       0.90      0.90      0.90        30


Struktur Pohon Keputusan:

|--- Petal Length Category <= 2.50
|   |--- Petal Width Category <= 1.50
|   |   |--- Petal Width Category <= 0.50
|   |   |   |--- Sepal Width Category <= 0.50
|   |   |   |   |--- class: 1
|   |   |   |--- Sepal Width Category >  0.50
|   |   |   |   |--- Petal Length Category <= 1.00
|   |   |   |   |   |--- Sepal Width Category <= 1.50
|   |   |   |   |   |   |--- Sepal Length Category <= 0.50
|   |   |   |   |   |   |   |--- class: 2
|   |   |   |   |   |   |--- Sepal Length Category >  0.50
|   |   |   |   |   |   |   |--- class:

## Diskritisasi Menggunakan Equal Width

In [ ]:
fitur = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']

def equal_width_all(features, n_bins=4):
    for col in features:
        min_val = df[col].min()
        max_val = df[col].max()
        bins = np.linspace(min_val, max_val, n_bins + 1)
        labels = [chr(97+i) for i in range(n_bins)]  # ['a', 'b', 'c', 'd']
        df[f'{col} EW Category'] = pd.cut(df[col], bins=bins, labels=labels, include_lowest=True)

# Jalankan fungsi
equal_width_all(fitur)

# Tampilkan hasil kategori huruf saja
print(tabulate(df[[f'{col} EW Category' for col in fitur]], headers='keys', tablefmt='github', showindex=False))

| Sepal Length EW Category   | Sepal Width EW Category   | Petal Length EW Category   | Petal Width EW Category   |
|----------------------------|---------------------------|----------------------------|---------------------------|
| a                          | c                         | a                          | a                         |
| a                          | b                         | a                          | a                         |
| a                          | b                         | a                          | a                         |
| a                          | b                         | a                          | a                         |
| a                          | c                         | a                          | a                         |
| b                          | d                         | a                          | a                         |
| a                          | c                         | a            

### Menggunakan Naive Bayes

In [ ]:

# -----------------------------
# Proses Klasifikasi Naive Bayes (CategoricalNB) dengan Equal Frequency
# -----------------------------

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Load dataset


# -----------------------------
# Equal Frequency Discretization (4 bin, huruf a–d)
# -----------------------------
fitur = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']
for col in fitur:
    labels = ['a', 'b', 'c', 'd']
    df[col] = pd.qcut(df[col], q=4, labels=labels, duplicates='drop')

# -----------------------------
# Encode kategori a–d jadi angka
# -----------------------------
le = LabelEncoder()
X = df[fitur].apply(le.fit_transform)
y = LabelEncoder().fit_transform(df['Species'])

# -----------------------------
# Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------
# Latih model Naive Bayes
# -----------------------------
model = CategoricalNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# -----------------------------
# Evaluasi
# -----------------------------
print("Akurasi:", accuracy_score(y_test, y_pred))
print("\nLaporan Klasifikasi:\n", classification_report(y_test, y_pred))

Akurasi: 0.9333333333333333

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.89      0.89      0.89         9
           2       0.91      0.91      0.91        11

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



### Menggunakan Decision Tree

In [ ]:

# -----------------------------
# Proses Klasifikasi Decision Tree dengan Equal Frequency
# -----------------------------

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.datasets import load_iris # Import load_iris again
from tabulate import tabulate # Import tabulate again

# Load dataset Iris again to get the original numerical data
iris = load_iris()
# Create a fresh DataFrame for this section
df_dt_ef = pd.DataFrame(iris.data, columns=['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width'])
df_dt_ef['Species'] = pd.Categorical.from_codes(iris.target, iris.target_names)


# -----------------------------
# Equal Frequency Discretization (4 bin, a–d) on the fresh DataFrame
# -----------------------------
fitur = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']
for col in fitur:
    # Apply qcut to the numerical columns in the fresh DataFrame
    df_dt_ef[f'{col}_EF_Category'] = pd.qcut(df_dt_ef[col], q=4, labels=['a', 'b', 'c', 'd'], duplicates='drop')

# Select the newly created categorical columns for the Decision Tree model
categorical_features_dt = [f'{col}_EF_Category' for col in fitur]


# -----------------------------
# Encode kategori huruf ke angka
# -----------------------------
le = LabelEncoder()
# Apply encoding to the new categorical columns
X = df_dt_ef[categorical_features_dt].apply(le.fit_transform)
# Encode target label from the fresh DataFrame
y = LabelEncoder().fit_transform(df_dt_ef['Species'])

# -----------------------------
# Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------
# Latih model Decision Tree
# -----------------------------
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# -----------------------------
# Evaluasi
# -----------------------------
print("Akurasi:", accuracy_score(y_test, y_pred))
print("\nLaporan Klasifikasi:\n", classification_report(y_test, y_pred))

# -----------------------------
# Cetak Struktur Pohon Keputusan
# -----------------------------
print("\nStruktur Pohon Keputusan:\n")
# Use the feature names from the encoded categorical columns
print(export_text(model, feature_names=categorical_features_dt))

Akurasi: 0.9333333333333333

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.89      0.89      0.89         9
           2       0.91      0.91      0.91        11

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30


Struktur Pohon Keputusan:

|--- Petal Length_EF_Category <= 0.50
|   |--- class: 0
|--- Petal Length_EF_Category >  0.50
|   |--- Petal Length_EF_Category <= 2.50
|   |   |--- Petal Width_EF_Category <= 2.50
|   |   |   |--- Sepal Width_EF_Category <= 2.50
|   |   |   |   |--- Petal Width_EF_Category <= 1.50
|   |   |   |   |   |--- Sepal Width_EF_Category <= 1.50
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- Sepal Width_EF_Category >  1.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- Petal Width_EF_Category >  1.50
|   |   |   | 

## Diskritisasi Menggunakan Equalfrecuency

In [ ]:

# Fungsi manual untuk equal-frequency discretization
def discretize_cdf(data, m):
    # Ensure data is numerical before sorting and calculations
    # Convert data to float, coercing errors to NaN if necessary (though Iris has no NaNs)
    data_numeric = pd.to_numeric(data, errors='coerce').dropna().tolist()

    data_sorted = sorted(data_numeric)
    n = len(data_sorted)
    thresholds = [-float('inf')]

    # Hitung batas kuantil
    for k in range(1, m):
        q = k / m
        index = (n - 1) * q
        floor = int(index)
        ceil = floor + 1
        frac = index - floor
        if ceil >= n:
            val = data_sorted[floor]
        else:
            # Ensure arithmetic operations are on numbers
            val = data_sorted[floor] + (data_sorted[ceil] - data_sorted[floor]) * frac
        thresholds.append(val)

    thresholds.append(float('inf'))

    # Tentukan label bin untuk setiap nilai
    labels = []
    label_map = {i: chr(97+i) for i in range(m)} # Define label_map

    # Apply the thresholds to the original data series
    # Use pd.cut to bin the original numerical data based on calculated thresholds
    binned_data = pd.cut(data, bins=thresholds, labels=label_map.values(), include_lowest=True)

    return binned_data.tolist() # Return as list to match original function output type

# Terapkan discretization ke setiap kolom
# Load dataset Iris again to get the original numerical data
# This is important because previous cells modified the 'df' DataFrame in place.
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris()
df_fresh = pd.DataFrame(iris.data, columns=['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width'])
df_fresh['Species'] = pd.Categorical.from_codes(iris.target, iris.target_names)


df_equal_frequency = pd.DataFrame({
    'sepal_length': discretize_cdf(df_fresh['Sepal Length'], 4),
    'sepal_width':  discretize_cdf(df_fresh['Sepal Width'], 4),
    'petal_length': discretize_cdf(df_fresh['Petal Length'], 4),
    'petal_width':  discretize_cdf(df_fresh['Petal Width'], 4)
})

# Tambahkan kolom kelas di depan
df_equal_frequency['Species'] = df_fresh['Species']


# Tampilkan hasil
print(df_equal_frequency.to_string(index=False))

sepal_length sepal_width petal_length petal_width    Species
           a           d            a           a     setosa
           a           b            a           a     setosa
           a           c            a           a     setosa
           a           c            a           a     setosa
           a           d            a           a     setosa
           b           d            b           b     setosa
           a           d            a           a     setosa
           a           d            a           a     setosa
           a           b            a           a     setosa
           a           c            a           a     setosa
           b           d            a           a     setosa
           a           d            a           a     setosa
           a           b            a           a     setosa
           a           b            a           a     setosa
           b           d            a           a     setosa
           b           d

### Menggunakan Naive Bayes

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, classification_report

# -----------------------------
# Encode fitur kategorikal (a–d) ke angka
# -----------------------------
X = df_equal_frequency.drop(columns='Species').apply(lambda col: LabelEncoder().fit_transform(col))
y = LabelEncoder().fit_transform(df_equal_frequency['Species'])

# -----------------------------
# Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------
# Latih model Naive Bayes
# -----------------------------
model = CategoricalNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# -----------------------------
# Evaluasi
# -----------------------------
print("Akurasi:", accuracy_score(y_test, y_pred))
print("\nLaporan Klasifikasi:\n", classification_report(y_test, y_pred))

Akurasi: 0.9333333333333333

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.89      0.89      0.89         9
           2       0.91      0.91      0.91        11

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30



### Menggunakan Decision Tree

In [ ]:
# -----------------------------
# Proses Klasifikasi Decision Tree dengan Equal Frequency
# -----------------------------

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
from sklearn.datasets import load_iris # Import load_iris again

# Load dataset Iris again to get the original numerical data
iris = load_iris()
# Create a fresh DataFrame for this section
df_dt_ef_final = pd.DataFrame(iris.data, columns=['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width'])
df_dt_ef_final['Species'] = pd.Categorical.from_codes(iris.target, iris.target_names)


# -----------------------------
# Diskritisasi Equal Frequency (4 bin: a–d)
# -----------------------------
fitur = ['Sepal Length', 'Sepal Width', 'Petal Length', 'Petal Width']
for col in fitur:
    # Apply qcut to the numerical columns in the fresh DataFrame
    df_dt_ef_final[f'{col}_EF_Category'] = pd.qcut(df_dt_ef_final[col], q=4, labels=['a', 'b', 'c', 'd'], duplicates='drop')

# Select the newly created categorical columns for the Decision Tree model
categorical_features_dt_final = [f'{col}_EF_Category' for col in fitur]


# -----------------------------
# Encode kategori huruf ke angka
# -----------------------------
le = LabelEncoder()
# Apply encoding to the new categorical columns
X = df_dt_ef_final[categorical_features_dt_final].apply(le.fit_transform)
# Encode target label from the fresh DataFrame
y = LabelEncoder().fit_transform(df_dt_ef_final['Species'])

# -----------------------------
# Train-test split
# -----------------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------
# Latih model Decision Tree
# -----------------------------
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# -----------------------------
# Evaluasi
# -----------------------------
print("Akurasi:", accuracy_score(y_test, y_pred))
print("\nLaporan Klasifikasi:\n", classification_report(y_test, y_pred))

# -----------------------------
# Cetak Struktur Pohon Keputusan
# -----------------------------
print("\nStruktur Pohon Keputusan:\n")
# Use the feature names from the encoded categorical columns
print(export_text(model, feature_names=categorical_features_dt_final))

Akurasi: 0.9333333333333333

Laporan Klasifikasi:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       0.89      0.89      0.89         9
           2       0.91      0.91      0.91        11

    accuracy                           0.93        30
   macro avg       0.93      0.93      0.93        30
weighted avg       0.93      0.93      0.93        30


Struktur Pohon Keputusan:

|--- Petal Length_EF_Category <= 0.50
|   |--- class: 0
|--- Petal Length_EF_Category >  0.50
|   |--- Petal Length_EF_Category <= 2.50
|   |   |--- Petal Width_EF_Category <= 2.50
|   |   |   |--- Sepal Width_EF_Category <= 2.50
|   |   |   |   |--- Petal Width_EF_Category <= 1.50
|   |   |   |   |   |--- Sepal Width_EF_Category <= 1.50
|   |   |   |   |   |   |--- class: 1
|   |   |   |   |   |--- Sepal Width_EF_Category >  1.50
|   |   |   |   |   |   |--- class: 0
|   |   |   |   |--- Petal Width_EF_Category >  1.50
|   |   |   | 

## Perbandingan Akurasi

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Fungsi evaluasi untuk 2 model
def evaluate_models(df, label_column='Species'):
    # Select only categorical columns for encoding, excluding the label column
    categorical_cols = [col for col in df.columns if col != label_column]
    X = df[categorical_cols].apply(lambda col: LabelEncoder().fit_transform(col.astype(str))) # Encode as string to handle potential mixed types from pandas cut/qcut
    y = LabelEncoder().fit_transform(df[label_column])
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Naive Bayes
    nb_model = CategoricalNB()
    nb_model.fit(X_train, y_train)
    nb_pred = nb_model.predict(X_test)
    acc_nb = accuracy_score(y_test, nb_pred)

    # Decision Tree
    dt_model = DecisionTreeClassifier(random_state=42)
    dt_model.fit(X_train, y_train)
    dt_pred = dt_model.predict(X_test)
    acc_dt = accuracy_score(y_test, dt_pred)


    return acc_nb, acc_dt

# ========================
# Asumsikan kamu sudah punya:
# - df_kmeans (kolom: 'Sepal Length_EF', ..., 'Species')
# - df_equal_width (dari pd.cut)
# - df_equal_frequency (dari discretize_cdf)
# ========================

# --- 1. KMeans ---
df_kmeans_nb = df_kmeans[[  # pilih hanya kolom kategori + kelas
    'Sepal Length_EF',
    'Sepal Width_EF',
    'Petal Length_EF',
    'Petal Width_EF',
    'Species'
]].rename(columns={
    'Sepal Length_EF': 'sepal_length',
    'Sepal Width_EF': 'sepal_width',
    'Petal Length_EF': 'petal_length',
    'Petal Width_EF': 'petal_width'
})
acc_kmeans_nb, acc_kmeans_dt = evaluate_models(df_kmeans_nb)

# --- 2. Equal Width ---
# Select only the categorical columns from df_equal_width
df_ew_nb = df_equal_width[['Sepal Length_EW_Category', 'Sepal Width_EW_Category', 'Petal Length_EW_Category', 'Petal Width_EW_Category', 'Species']].copy()
acc_ew_nb, acc_ew_dt = evaluate_models(df_ew_nb)

# --- 3. Equal Frequency ---
# Select only the categorical columns from df_equal_frequency
df_ef_nb = df_equal_frequency[['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'Species']].copy()
acc_ef_nb, acc_ef_dt = evaluate_models(df_ef_nb)


# ========================
# Tampilkan hasil perbandingan lengkap
# ========================
print("\n📊 Perbandingan Akurasi (Naive Bayes & Decision Tree):\n")
print(f"{'Metode':<25} {'Naive Bayes':>12} {'Decision Tree':>18}")
print("-" * 55)
print(f"{'KMeans Clustering':<25} {acc_kmeans_nb:>12.4f} {acc_kmeans_dt:>18.4f}")
print(f"{'Equal Width (cut)':<25} {acc_ew_nb:>12.4f} {acc_ew_dt:>18.4f}")
print(f"{'Equal Frequency (CDF)':<25} {acc_ef_nb:>12.4f} {acc_ef_dt:>18.4f}")


📊 Perbandingan Akurasi (Naive Bayes & Decision Tree):

Metode                     Naive Bayes      Decision Tree
-------------------------------------------------------
KMeans Clustering               0.9333             0.9333
Equal Width (cut)               0.9333             0.9667
Equal Frequency (CDF)           0.9333             0.9333
